In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
import sys
import random
import pyautogui
from PyQt5 import QtWidgets, uic
import icon_rc_
import webbrowser

In [2]:
# LSTM for international airline passengers problem with memory
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import time
import csv #엑셀파일
import os
import win32com.client 
from openpyxl import Workbook
import xlwt
import math
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import openpyxl 
import matplotlib.pyplot as plt
import keras
from keras.models import load_model

Using TensorFlow backend.


In [3]:
basic_ui = uic.loadUiType("UI_design.ui")[0]

In [4]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [],[] 
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back):]
        dataX.append(a)
        dataY.append(dataset[i:(i + look_back):])
    
    dataX=np.squeeze(dataX,axis=1) #차원 축소
    dataY=np.squeeze(dataY,axis=1) #차원 축소
    return np.array(dataX), np.array(dataY)

In [5]:
np.random.seed(7)

data1=pd.read_excel('x_data.xlsx') #x좌표 엑셀 데이터 불러옴 
data2=pd.read_excel('y_data.xlsx') #y좌표 엑셀 데이터 불러옴 
data3=pd.read_excel('speed_data.xlsx') #speed좌표 엑셀 데이터 불러옴 

pddata1=pd.DataFrame(data1) 
pddata1.head()
pddata2=pd.DataFrame(data2) 
pddata2.head()
pddata3=pd.DataFrame(data3) 
pddata3.head()

hap=[] #진짜 합
for j in range(len(pddata1)):
    x_data=np.array(pddata1.loc[j])
    y_data=np.array(pddata2.loc[j])
    s_data=np.array(pddata3.loc[j])

    hap1=[] #[x,y,속력] 데이터를 저장하고 있는 리스트
    
    for i in range(len(s_data)): 
        sum=[] #리스트 하나당 임시로 x,y,속력을 저장할 리스트 (for문 돌릴때마다 초기화)
        sum.append(x_data[i]) #x넣음
        sum.append(y_data[i]) #y넣음
        sum.append(s_data[i]) #속력넣음
        hap1.append(sum) #[x,y,속력] 하나의 리스트를 hap리스트에 넣음  

    j = j + 1
    
    hap.append(hap1)

n1 = np.array(hap)
print(n1.shape)

(5502, 99, 3)


In [6]:
num_sample = n1.shape[0] # 257개 Sample 데이터
num_sequence = n1.shape[1] # 99개 시계열 데이터
num_feature = n1.shape[2] #3개 Feature

dataset = n1

In [7]:
train_size = int(len(dataset) * 0.70) #학습 데이터 70%
test_size = len(dataset) - train_size #테스트 데이터 30%
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [8]:
look_back = 1 #이전 시간 단계 입력변수
#X는 지금 t 값이고, Y는 그 다음의 t+1 값임 (즉, X=t, Y=t+1)
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [9]:
from keras.layers import TimeDistributed
from keras.layers import InputLayer
from keras.layers import Reshape
from keras.layers import *
from keras.models import *
from keras.utils import *

model = Sequential() # Sequeatial Model 
model.add(LSTM(32,return_sequences=True, input_shape=(num_sequence,num_feature)))  
model.add(Dense(3, activation='softmax')) # 3개의 예측
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

history = model.fit(trainX, trainY, epochs=10, batch_size=20, validation_split=0.2, verbose=1)

model.save('lstm_mouseAI.h5')

Train on 3079 samples, validate on 770 samples
Epoch 1/10
3079/3079 [==============================] - 13s 4ms/step - loss: 1106.6588 - accuracy: 0.8825 - val_loss: 916.7249 - val_accuracy: 1.0000
Epoch 2/10
3079/3079 [==============================] - 13s 4ms/step - loss: 898.3457 - accuracy: 1.0000 - val_loss: 874.5332 - val_accuracy: 1.0000
Epoch 3/10
3079/3079 [==============================] - 13s 4ms/step - loss: 874.4777 - accuracy: 1.0000 - val_loss: 862.2047 - val_accuracy: 1.0000
Epoch 4/10
3079/3079 [==============================] - 13s 4ms/step - loss: 865.9091 - accuracy: 1.0000 - val_loss: 856.9795 - val_accuracy: 1.0000
Epoch 5/10
3079/3079 [==============================] - 13s 4ms/step - loss: 861.8007 - accuracy: 1.0000 - val_loss: 854.1235 - val_accuracy: 1.0000
Epoch 6/10
3079/3079 [==============================] - 13s 4ms/step - loss: 859.5255 - accuracy: 1.0000 - val_loss: 852.5949 - val_accuracy: 1.0000
Epoch 7/10
3079/3079 [==============================] - 13

In [10]:
print("정확도: ",(model.evaluate(testX, testY)))

1649/1649 [==============================] - 2s 919us/step
정확도:  [906.892002010866, 1.0]


In [4]:
#softmax 역정규화 
def inverse_softmax(s, x): #정규화된 값, test값
    e_x = np.exp(x)
    return np.log(s) + np.log(np.sum(e_x))

In [4]:
#버튼 출력 윈도우창   
class WindowClass(QMainWindow, basic_ui):
    def __init__(self):
        super().__init__()
        self.setupUi(self)

        self.cnt = 0
        
        self.x_data=[]
        self.y_data=[]
        self.speed=[]
        self.n = 0

        self.show()
        
        qr = self.frameGeometry()
        cp = QDesktopWidget().availableGeometry().center()
        qr.moveCenter(cp)
        self.move(qr.topLeft())
        
        self.kwangwoon_btn.clicked.connect(self.ClickMethod)
        self.naver_btn.clicked.connect(self.ClickMethod)
        self.google_btn.clicked.connect(self.ClickMethod)
        self.youtube_btn.clicked.connect(self.ClickMethod)
        self.instagram_btn.clicked.connect(self.ClickMethod)
        self.twitter_btn.clicked.connect(self.ClickMethod)
        self.zoom_btn.clicked.connect(self.ClickMethod)
        self.facebook_btn.clicked.connect(self.ClickMethod)
        self.everytime_btn.clicked.connect(self.ClickMethod)
        #self.start_btn.clicked.connect(self.createCsv)
        self.start_btn.clicked.connect(self.Start) #최종 프로젝트 
        self.show()

    def createCsv(self):

            #엑셀파일에 x좌표값 저장
            sheet1 = ["Sheet 1"]

            wb = xlwt.Workbook()

            for sheet, data in zip(sheet1, [self.x_data]):
                ws = wb.add_sheet(sheet)

                for row, row_value in enumerate(data):
                    for col, col_value in enumerate(row_value):
                        ws.write(row, col, col_value)

            wb.save('x_data.xls')

            #엑셀파일에 y좌표값 저장
            sheet2 = ["Sheet 1"]

            wb = xlwt.Workbook()

            for sheet, data in zip(sheet2, [self.y_data]):
                ws = wb.add_sheet(sheet)

                for row, row_value in enumerate(data):
                    for col, col_value in enumerate(row_value):
                        ws.write(row, col, col_value)

            wb.save('y_data.xls')
            
            #엑셀파일에 속력값 저장
            sheet3 = ["Sheet 1"]

            wb = xlwt.Workbook()

            for sheet, data in zip(sheet3, [self.speed]):
                ws = wb.add_sheet(sheet)

                for row, row_value in enumerate(data):
                    for col, col_value in enumerate(row_value):
                        ws.write(row, col, col_value)

            wb.save('speed.xls')
            

            #x좌표와 y좌표 화면 상 확인
            for x_Data in self.x_data:
                print(x_Data)
            for y_Data in self.y_data:
                print(y_Data)
            for speed_Data in self.speed:
                print(speed_Data)
    
    def ClickMethod(self):

            for j in range(100):
                x,y=pyautogui.position()
                print(x,y)
                time.sleep(0.001)
            # 안쪽 리스트로 사용할 빈 리스트 생성
            line_x = []
            line_y = []
            line_speed=[]

            for j in range(100):
                x,y=pyautogui.position()
                line_x.append(x)
                line_y.append(y)# 안쪽 리스트에 0 추가
                time.sleep(0.001)

            self.x_data.append(line_x)
            self.y_data.append(line_y)
            
            for i in range(1, len(line_x)):
                Speed=((line_x[i]-line_x[i-1])**2) + ((line_y[i]-line_y[i-1])**2)
                Speed=math.sqrt(Speed)
                line_speed.append(Speed)
            self.speed.append(line_speed)
            
    def Start(self):
        
            # 안쪽 리스트로 사용할 빈 리스트 생성
            line_x = []
            line_y = []
            line_speed=[]
        
            for j in range(100): #1초동안 데이터 수집 
                x,y=pyautogui.position()
                line_x.append(x)
                line_y.append(y)# 안쪽 리스트에 0 추가
                time.sleep(0.001)

            self.x_data.append(line_x)
            self.y_data.append(line_y)
            
            for i in range(len(line_x)):
                Speed=((line_x[i]-line_x[i-1])**2) + ((line_y[i]-line_y[i-1])**2)
                Speed=math.sqrt(Speed)
                line_speed.append(Speed)
            self.speed.append(line_speed)
            
            testSet=[]
            
            for i in range(1,len(line_x)):
            
                test=[] #리스트 하나당 임시로 x,y,속력 저장 리스트
                test.append(line_x[i])
                test.append(line_y[i])
                test.append(line_speed[i])
                testSet.append(test)
            
            Test=[]
            Test.append(testSet)
            
            Test_set = np.array(Test)
            
            print(Test_set)
            
            model=load_model('lstm_mouseAI.h5')
            testPredict = model.predict(Test_set)
        
            print("TestSet 값 출력 : ", testPredict)
            
            sum=Test_set[0,98,0]+Test_set[0,98,1]+Test_set[0,98,2]
            result_x=sum*testPredict[0,98,0]
            result_y=sum*testPredict[0,98,1]
            x_position=result_x #TestSet[0,98,0] #x값
            y_position=result_y #TestSet[0,98,1] #y값
            
            print("x y 값 출력 : ", x_position, y_position)
            n=0  #결과에따라 사이트 이동
            while(n==0):
                if x_position>830 and x_position<930 :
                    if 300<y_position and y_position<450:
                        #광운대
                        url = 'https://www.kw.ac.kr/ko/'
                        webbrowser.open(url)
                    if 400<y_position and y_position<550:
                        #유튜브
                        url = 'https://www.youtube.com/'
                        webbrowser.open(url)
                    if 550<y_position and y_position<700:
                        #인스타그램
                        url = 'https://www.instagram.com/?hl=ko'
                        webbrowser.open(url)
                        
                if 930<x_position and x_position<1030:
                    if 300<y_position and y_position<450:
                        #구글
                        url = 'https://www.google.co.kr/'
                        webbrowser.open(url)
                    if 400<y_position and y_position<550:
                        #에브리타임
                        url = 'https://everytime.kr/'
                        webbrowser.open(url)
                    if 550<y_position and y_position<700:
                        #트위터
                        url = 'https://twitter.com/?lang=ko'
                        webbrowser.open(url)
                        
                if 1030<x_position and x_position<1130:
                    if 300<y_position and y_position<450:
                        #네이버
                        url = 'https://www.naver.com/'
                        webbrowser.open(url)
                    if 400<y_position and y_position<550:
                        #페이스북
                        url = 'https://www.facebook.com/'
                        webbrowser.open(url)
                    if 550<y_position and y_position<700:
                        #zoom
                        url = 'https://zoom.us/'
                        webbrowser.open(url)
                        
                n+=1;

                              

app = QApplication([])
ex = WindowClass()
sys.exit(app.exec_())

[[[953.         672.           0.        ]
  [953.         672.           0.        ]
  [953.         672.           0.        ]
  [953.         672.           0.        ]
  [953.         672.           0.        ]
  [953.         672.           0.        ]
  [953.         671.           1.        ]
  [953.         671.           0.        ]
  [953.         671.           0.        ]
  [951.         671.           2.        ]
  [951.         671.           0.        ]
  [950.         670.           1.41421356]
  [950.         670.           0.        ]
  [950.         670.           0.        ]
  [949.         669.           1.41421356]
  [949.         669.           0.        ]
  [947.         669.           2.        ]
  [947.         669.           0.        ]
  [945.         667.           2.82842712]
  [945.         667.           0.        ]
  [945.         667.           0.        ]
  [944.         667.           1.        ]
  [944.         667.           0.        ]
  [942.    

SystemExit: 0

C:\Users\meid9\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
